In [1]:
from requests import get
import pandas as pd
from API_Keys import quandl
from datetime import datetime, timedelta

# To add new data sources from Quandl
Add your appropriately named URL in the cell below to the right group.

Add that variable to the URL list at the end of the correct group

Adjust range in cells that both merge them and also filter them to account for your addition

For the renaming of merged columns, add your renaming schemata to the correct area.

# This will fail if any step is messed up, or if the quandl data is organized differently than expected. This is hard to automate, potential issues can be expected.

Also, if we need to keep different columns for new data, this will also break.

In [2]:
# Group 1, keep Change, Settle, Volume, Previous Day Open Interest
CMEgold_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_GC1.json?api_key={quandl}"
CMEgold_futures2 = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_GC2.json?api_key={quandl}"
CMEsilver_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_SI1.json?api_key={quandl}"
CMEsilver_futures2 = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_SI2.json?api_key={quandl}"
CMEpalladium_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_PA1.json?api_key={quandl}"
CMEpalladium_futures2 = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_PA2.json?api_key={quandl}"
CMEplatinum_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_PL1.json?api_key={quandl}"
CMEplatinum_futures2 = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_PL2.json?api_key={quandl}"
CME_TenYear_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_TY1.json?api_key={quandl}"
CME_TenYear_futures2 = f"https://www.quandl.com/api/v3/datasets/CHRIS/CME_TY2.json?api_key={quandl}"

#Group 2, keep Change, Settle, Wave, Volume, Prev. Day Open Interest
ICE_USD_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/ICE_DX1.json?api_key={quandl}"
ICE_USD_futures2 = f"https://www.quandl.com/api/v3/datasets/CHRIS/ICE_DX2.json?api_key={quandl}"
ICE_ZAR_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/ICE_ZR1.json?api_key={quandl}"
ICE_ZAR_futures2 = f"https://www.quandl.com/api/v3/datasets/CHRIS/ICE_ZR2.json?api_key={quandl}"
# SGX_EuroD_futures = f"https://www.quandl.com/api/v3/datasets/CHRIS/SGX_ED1.json?api_key={quandl}"

#Group 3, keep Open Interest, last one on list also decides cut off date.
# Last one needs to be shortest weekly separation
Gold_OpenInt = f"https://www.quandl.com/api/v3/datasets/CFTC/088691_FO_ALL.json?api_key={quandl}"
Silver_OpenInt = f"https://www.quandl.com/api/v3/datasets/CFTC/084691_FO_ALL.json?api_key={quandl}"
Palladium_OpenInt = f"https://www.quandl.com/api/v3/datasets/CFTC/075651_FO_ALL.json?api_key={quandl}"
Platinum_OpenInt = f"https://www.quandl.com/api/v3/datasets/CFTC/076651_FO_ALL.json?api_key={quandl}"

In [3]:
url_list = [CMEgold_futures,CMEgold_futures2,CMEsilver_futures,CMEsilver_futures2,CMEpalladium_futures,CMEpalladium_futures2,CMEplatinum_futures,CMEplatinum_futures2,CME_TenYear_futures,CME_TenYear_futures2,ICE_USD_futures,ICE_USD_futures2,ICE_ZAR_futures,ICE_ZAR_futures2,Gold_OpenInt,Silver_OpenInt,Palladium_OpenInt,Platinum_OpenInt]

In [4]:
info_list = [get(url).json() for url in url_list]

In [43]:
df_list = [pd.DataFrame(info["dataset"]["data"],columns=info["dataset"]["column_names"]) for info in info_list]
for i in range(1,6):
    df_list[i-1]["Previous Day Open Interest"] += df_list[i]["Previous Day Open Interest"]
    df_list.pop(i)
for i in range(6,8):
    df_list[i-1]["Prev. Day Open Interest"] += df_list[i]["Prev. Day Open Interest"]
    df_list.pop(i)

In [44]:
# Creates a list that functions well in testing
pruned_list = [None]*11

In [45]:
# Cell to prune Group 1
metal_namer = {0:"Gold",1:"Silver",2:"Palladium",3:"Platinum",4:"Ten_Yr_Futures"}
for i in range(5):
    pruned_list[i] = df_list[i][["Date","Change","Settle","Volume","Previous Day Open Interest"]]
    pruned_list[i].columns = ["Date",f"{metal_namer[i]}_Change",f"{metal_namer[i]}_Settle",f"{metal_namer[i]}_Volume",f"{metal_namer[i]}_Prev. Day Open Interest"]

In [46]:
# Cell to prune Group 2
money_namer = {5:"USD",6:"ZAR"}
for i in range(5,7):
    pruned_list[i] = df_list[i][["Date","Change","Settle","Volume","Wave","Prev. Day Open Interest"]]
    pruned_list[i].columns = ["Date",f"{money_namer[i]}_Change",f"{money_namer[i]}_Settle",f"{money_namer[i]}_Volume",f"{money_namer[i]}_Wave",f"{money_namer[i]}_Prev. Day Open Interest"]

In [47]:
for i in range(7,11):
    pruned_list[i] = df_list[i][["Date","Open Interest","Money Manager Shorts","Money Manager Longs","Producer/Merchant/Processor/User Longs","Producer/Merchant/Processor/User Shorts"]]

In [72]:
# Only works for days that are Tuesday releases on their latest and most recent
# will need to add code to adjust for late releases because of Holidays
cut_off_date = df_list[len(df_list)-1]["Date"][len(df_list[len(df_list)-1].index)-1]
cut_off_date = (datetime.strptime(cut_off_date,"%Y-%m-%d") - timedelta(days=6))
cut_off_date = cut_off_date.strftime("%Y-%m-%d")
# recent_off_date = df_list[len(df_list)-1]["Date"][0]
# recent_off_date = (datetime.strptime(recent_off_date,"%Y-%m-%d") - timedelta(days=4))
# recent_off_date = recent_off_date.strftime("%Y-%m-%d")

In [73]:
cut_off_date

'2006-06-07'

In [50]:
for i in range(7):
    pruned_list[i] = pruned_list[i].loc[pruned_list[i]["Date"]>=cut_off_date]
#     pruned_list[i] = pruned_list[i].loc[pruned_list[i]["Date"]<=recent_off_date]

In [65]:
grouped_list = [None]*7
tuple_list = [[(datetime.strptime(cut_off_date,"%Y-%m-%d")+timedelta(days=(w*7))),(datetime.strptime(cut_off_date,"%Y-%m-%d")+timedelta(days=(w*7 + 6)))] for w in range(len(df_list[len(df_list)-1].index))]
# tuple_list = [x if x[0].weekday() == 1 else [x[0]+timedelta(days=1),x[1]+timedelta(days=1)] for x in tuple_list]
# tuple_list = [tuple(x) if x[0].weekday() == 1 else (x[0]+timedelta(days=1),x[1]+timedelta(days=1)) for x in tuple_list]
# cut_bins = pd.IntervalIndex.from_tuples(tuple_list,closed="both")

In [71]:
tuple_list[0][1]

datetime.datetime(2006, 6, 10, 0, 0)

In [58]:
for bin_candidate in range(7):
    agg_dic = {column:("sum" if "change" in column.lower() else "mean") for column in list(pruned_list[bin_candidate].columns)}
    agg_dic.pop("Date")
    bins = pd.cut(pruned_list[bin_candidate]["Date"],cut_bins)
    grouped_list[bin_candidate] = pruned_list[bin_candidate].groupby(bins).agg(agg_dic)
    grouped_list[bin_candidate] = grouped_list[bin_candidate].assign(Date=list(pruned_list[len(pruned_list)-1]["Date"].iloc[::-1]))
    grouped_list[bin_candidate].index.names = ["Business_Week"]

In [59]:
merged_list = [grouped_list[i].\
    merge(pruned_list[7+i], on = "Date", how = "inner").\
    merge(grouped_list[4], on = "Date", how = "inner").\
    merge(grouped_list[5], on = "Date", how = "inner").\
    merge(grouped_list[6], on = "Date", how = "inner").\
    set_index(grouped_list[i].index) for i in range(4)]

In [60]:
merged_list[0].head(1)

,Gold_Change,Gold_Settle,Gold_Volume,Gold_Prev. Day Open Interest,Date,Open Interest,Money Manager Shorts,Money Manager Longs,Producer/Merchant/Processor/User Longs,Producer/Merchant/Processor/User Shorts,...,USD_Change,USD_Settle,USD_Volume,USD_Wave,USD_Prev. Day Open Interest,ZAR_Change,ZAR_Settle,ZAR_Volume,ZAR_Wave,ZAR_Prev. Day Open Interest
Business_Week,,,,,,,,,,,,,,,,,,,,,
"[2006-06-06, 2006-06-12]",0.0,616.18,355.6,189796.6,2006-06-13,382474.0,30916.0,103211.0,42587.0,128892.0,...,1.74,85.41,7822.0,NaN,39002.4,0.1105,6.7638,63.8,NaN,2325.8


In [35]:
len(merged_list[0].dropna().index)

136

In [36]:
merged_list[0].to_csv("../data/csv/Gold_Data.csv")
merged_list[1].to_csv("../data/csv/Silver_Data.csv")
merged_list[2].to_csv("../data/csv/Palladium_Data.csv")
merged_list[3].to_csv("../data/csv/Platinum_Data.csv")